In [1]:
%defaultDatasource jdbc:postgresql://aquamarin.inttech.flab.fujitsu.co.jp/mimic?user=ugai&password=Love2Honami

# コホートテーブルの作成　(my_cohort)

-17歳より大きい
-48時間以上滞在
-出血がない
-透析していない (procedures_icd から取っているので、請求書ベース)
-輸血していない (procedures_icd から取っているので、請求書ベース)
-手術していない (procedures_icd から取っているので、請求書ベース)

In [2]:
set search_path to mimiciii ;
DROP TABLE IF EXISTS  my_cohort ;
CREATE TABLE my_cohort AS
WITH co0 AS -- 最初の入院の最初のICUのID
(
select MIN(icustays.intime) as intime, subject_id
from icustays
GROUP BY (subject_id)
), co AS 
(
select icustays.*,patients.gender, EXTRACT('year' from age(icustays.intime,patients.dob)) as age
from icustays, co0, patients
where icustays.intime = co0.intime
AND icustays.subject_id = co0.subject_id
AND icustays.subject_id = patients.subject_id
),
cohort AS (
select co.*, 
  CASE WHEN co.age > 17 AND co.age < 100 THEN TRUE
           ELSE FALSE
      END as d_age,
    CASE WHEN co.los >= 2.0 THEN TRUE
           ELSE FALSE
      END as d_los,
    CASE WHEN co.hadm_id in (
            select distinct hadm_id
            from diagnoses_icd as icd
            where 
                icd.icd9_code in('5310', '5312', '5314', '5316', '5320', '5322', '5324', '5326', '5330', '5332', '5334', '5336', '5340', '5342', '5344', '5346', '5780', '5781', '5789', '5693')
            ) THEN FALSE
           ELSE TRUE
      END as d_bleed,
    CASE WHEN co.hadm_id in (
            select distinct proc.hadm_id
            from procedures_icd as proc
            where proc.icd9_code = '3995'
            ) THEN FALSE
            ELSE TRUE
       END as d_hd,
     CASE WHEN co.hadm_id in (
            select distinct proc.hadm_id
            from procedures_icd as proc
            where proc.icd9_code in ('9901', '9902', '9903', '9904', '9905', '9906', '9907', '9908', '9909')
            ) THEN FALSE
            ELSE TRUE
        END as d_transfusion,
     CASE WHEN co.hadm_id in (
             select distinct proc.hadm_id
            from procedures_icd as proc, d_icd_procedures as dic
            where proc.icd9_code = dic.icd9_code
            AND dic.long_title LIKE '%urg%' 
            ) THEN FALSE
            ELSE TRUE
        END as d_surgery
    from co
)
select cohort.*
from cohort
WHERE d_age AND d_los AND d_bleed AND d_hd AND d_transfusion AND d_surgery


# t1(一番最初のヘモグロビンの測定)つきコホートテーブルの作成　(my_cohort_t1)

In [3]:
set search_path to mimiciii ;
DROP TABLE IF EXISTS  my_cohort_t1 ;
CREATE TABLE my_cohort_t1 AS
WITH first AS
(
select MIN(labevents.charttime) as first, labevents.hadm_id
from labevents, my_cohort as co
where 
    co.hadm_id = labevents.hadm_id
    AND co.intime < labevents.charttime
    AND co.outtime > labevents.charttime
    AND labevents.itemid = 51222
GROUP BY (labevents.hadm_id)
)
select co.*, first.first, labevents.valuenum as t1value
from my_cohort as co, first, labevents
where co.hadm_id = first.hadm_id
AND first.hadm_id = labevents.hadm_id
AND labevents.itemid = 51222
AND labevents.charttime = first.first

一番最初のヘモグロビンの測定と、24時間後で一番早いヘモグロビンの測定

In [11]:
set search_path to mimiciii ;
DROP TABLE IF EXISTS  my_cohort_t1_t2 ;
CREATE TABLE my_cohort_t1_t2 AS
WITH t2 AS
(
select MIN(labevents.charttime) as t2, labevents.hadm_id
from labevents, my_cohort_t1 as co
where
    co.hadm_id = labevents.hadm_id
    AND co.first  + interval '24 hours'< labevents.charttime
    AND co.outtime > labevents.charttime
    AND labevents.itemid = 51222
GROUP BY (labevents.hadm_id)
)
select co.*,t2.t2,labevents.valuenum as t2value, labevents.valuenum-co.t1value as deltaHemo, EXTRACT (EPOCH from (t2.t2-co.first))/60/60 as T12H, 24*3600/EXTRACT (EPOCH from (t2.t2-co.first)) as timescale
from t2, my_cohort_t1 as co, labevents
where co.hadm_id = t2.hadm_id
AND t2.hadm_id = labevents.hadm_id
AND labevents.itemid = 51222
AND labevents.charttime = t2.t2

T1,T2の間の尿の量を追加, outcome の追加 (outcome: 1が死亡、０が退院）

In [12]:
set search_path to mimiciii ;
DROP TABLE IF EXISTS  my_cohort_t1_t2_urineoutput ;
CREATE TABLE my_cohort_t1_t2_urineoutput AS
WITH urine AS
(
select urineoutput.icustay_id, sum(urineoutput.value) as urinevalue
from my_cohort_t1_t2 as co, urineoutput
where co.icustay_id = urineoutput.icustay_id
AND co.first < urineoutput.charttime
AND co.t2 > urineoutput.charttime
group by urineoutput.icustay_id
)
select co.*, urine.urinevalue, admissions.HOSPITAL_EXPIRE_FLAG as outcome
from my_cohort_t1_t2 as co, urine, admissions
where co.icustay_id = urine.icustay_id
AND co.hadm_id = admissions.hadm_id

In [13]:
set search_path to mimiciii;
DROP TABLE IF EXISTS my_cohort_t1_t2_urineoutput_fluid;
CREATE TABLE my_cohort_t1_t2_urineoutput_fluid AS
with t1 as
(
  select
    co.icustay_id
  , round(case
      when mv.amountuom = 'L'
        then mv.amount * 1000.0
      when mv.amountuom = 'ml'
        then mv.amount
    else null end) as amount
  from my_cohort_t1_t2 as co
  inner join inputevents_mv mv
  on co.icustay_id = mv.icustay_id
  and mv.itemid in
  (
    -- 225943 Solution
    225158, -- NaCl 0.9%
    225828, -- LR
    225944, -- Sterile Water
    225797  -- Free Water
  )
  where mv.statusdescription != 'Rewritten'
  and
  -- in MetaVision, these ITEMIDs appear with a null rate IFF endtime=starttime + 1 minute
  -- so it is sufficient to:
  --    (1) check the rate is > 240 if it exists or
  --    (2) ensure the rate is null and amount > 240 ml
    (
      (mv.rate is not null and mv.rateuom = 'mL/hour' )
      OR (mv.rate is not null and mv.rateuom = 'mL/min' )
      OR (mv.rate is null and mv.amountuom = 'L' )
      OR (mv.rate is null and mv.amountuom = 'ml' )
    )
    AND mv.starttime > co.first AND mv.starttime < co.t2
)
, t2 as
(
  select
    co.icustay_id
  , -- carevue always has units in millilitres
   round(cv.amount) as amount
  from my_cohort_t1_t2 as co
  inner join inputevents_cv cv
  on co.icustay_id = cv.icustay_id
  and cv.itemid in
  (
    30018 --    .9% Normal Saline
  , 30021 --    Lactated Ringers
  , 30058 --    Free Water Bolus
  , 40850 --    ns bolus
  , 41491 --    fluid bolus
  , 42639 --    bolus
  , 30065 --    Sterile Water
  , 42187 --    free h20
  , 43819 --    1:1 NS Repletion.
  , 30063 --    IV Piggyback
  , 41430 --    free water boluses
  , 40712 --    free H20
  , 44160 --    BOLUS
  , 42383 --    cc for cc replace
  , 30169 --    Sterile H20_GU
  , 42297 --    Fluid bolus
  , 42453 --    Fluid Bolus
  , 40872 --    free water
  , 41915 --    FREE WATER
  , 41490 --    NS bolus
  , 46501 --    H2O Bolus
  , 45045 --    WaterBolus
  , 41984 --    FREE H20
  , 41371 --    ns fluid bolus
  , 41582 --    free h20 bolus
  , 41322 --    rl bolus
  , 40778 --    Free H2O
  , 41896 --    ivf boluses
  , 41428 --    ns .9% bolus
  , 43936 --    FREE WATER BOLUSES
  , 44200 --    FLUID BOLUS
  , 41619 --    frfee water boluses
  , 40424 --    free H2O
  , 41457 --    Free H20 intake
  , 41581 --    Water bolus
  , 42844 --    NS fluid bolus
  , 42429 --    Free water
  , 41356 --    IV Bolus
  , 40532 --    FREE H2O
  , 42548 --    NS Bolus
  , 44184 --    LR Bolus
  , 44521 --    LR bolus
  , 44741 --    NS FLUID BOLUS
  , 44126 --    fl bolus
  , 44110 --    RL BOLUS
  , 44633 --    ns boluses
  , 44983 --    Bolus NS
  , 44815 --    LR BOLUS
  , 43986 --    iv bolus
  , 45079 --    500 cc ns bolus
  , 46781 --    lr bolus
  , 45155 --    ns cc/cc replacement
  , 43909 --    H20 BOlus
  , 41467 --    NS IV bolus
  , 44367 --    LR
  , 41743 --    water bolus
  , 40423 --    Bolus
  , 44263 --    fluid bolus ns
  , 42749 --    fluid bolus NS
  , 45480 --    500cc ns bolus
  , 44491 --    .9NS bolus
  , 41695 --    NS fluid boluses
  , 46169 --    free water bolus.
  , 41580 --    free h2o bolus
  , 41392 --    ns b
  , 45989 --    NS Fluid Bolus
  , 45137 --    NS cc/cc
  , 45154 --    Free H20 bolus
  , 44053 --    normal saline bolus
  , 41416 --    free h2o boluses
  , 44761 --    Free H20
  , 41237 --    ns fluid boluses
  , 44426 --    bolus ns
  , 43975 --    FREE H20 BOLUSES
  , 44894 --    N/s 500 ml bolus
  , 41380 --    nsbolus
  , 42671 --    free h2o
  )
  where 
    -- cv.amount > 248
  -- and cv.amount < 2000
  -- and
    cv.amountuom = 'ml'
    AND cv.charttime > co.first AND cv.charttime < co.t2
), t3 as
(
  select icustay_id, sum(amount) as crystalloid_bolus
  from t1
      -- just because the rate was high enough, does *not* mean the final amount was
  -- where amount > 248
  group by t1.icustay_id
  UNION
  select icustay_id, sum(amount) as crystalloid_bolus
  from t2
  group by t2.icustay_id
  order by icustay_id
) select icu.*, crystalloid_bolus as fluid
  from t3 inner join my_cohort_t1_t2_urineoutput icu
  on t3.icustay_id = icu.icustay_id

In [14]:
set search_path to mimiciii;
select count(*) from my_cohort_t1_t2_urineoutput_fluid

9375

In [20]:
set search_path to mimiciii;
select * from my_cohort_t1_t2_urineoutput_fluid limit 5

In [29]:
set search_path to mimiciii;
select co.gender,co.age, co.deltahemo/co.timescale as delta , co.urinevalue/co.timescale as urine, co.fluid/co.timescale as fluid, co.outcome
from my_cohort_t1_t2_urineoutput_fluid as co
limit 5